<a href="https://colab.research.google.com/github/dhanunjaireddy07/Sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pandas numpy matplotlib seaborn scikict-learn nltk wordcloud
!pip install emoji --upgrade
!pip install wordcloud
!pip install tensorflow
!pip install spacy
!python -m spacy download en_core_web_sm




import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from wordcloud import WordCloud

# NLTK downloads
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer


ERROR: Could not find a version that satisfies the requirement scikict-learn (from versions: none)
ERROR: No matching distribution found for scikict-learn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 118.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
# sentiment_analysis_optimized.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re, string, emoji, nltk, spacy

from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


import warnings
warnings.filterwarnings("ignore")

# NLTK + spaCy setup
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Load CSV
df = pd.read_csv("tweets.csv", header=None)
df.columns = ['tweet_id','entity', 'sentiment', 'text']
df.dropna(subset=['text', 'entity'], inplace=True)


# CLEANING PIPELINE
def clean_text(text):
    text = emoji.demojize(str(text))
    text = re.sub(r"http\S+|@\S+|#\S+|RT|[\d]+|[^\w\s]", "", text.lower())
    return text.strip()

def highlight_entity(text, entity):
    return re.sub(re.escape(entity.lower()), f"[ENTITY] {entity.lower()} [/ENTITY]", text, flags=re.IGNORECASE)

def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(w) for w in text.split() if w.isalpha()])

def preprocess_row(row):
    cleaned = clean_text(row["text"])
    highlighted = highlight_entity(cleaned, row["entity"])
    return lemmatize(highlighted)

df['clean_text'] = df.apply(preprocess_row, axis=1)
df = df[df['sentiment'].isin(['Positive', 'Negative', 'Neutral'])]
df['sentiment'] = df['sentiment'].str.lower()

# TF-IDF + CLASSICAL ML MODELS
X = df['clean_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("\n=== LOGISTIC REGRESSION ===")
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)
y_pred = lr.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

print("\n=== RANDOM FOREST TUNING (FAST) ===")
rf = RandomForestClassifier(random_state=42)
rf_params = {
    'n_estimators': [100],
    'max_depth': [None, 10],
    'min_samples_split': [2]
}
rf_search = RandomizedSearchCV(rf, rf_params, cv=3, n_iter=5, scoring='accuracy', n_jobs=-1, verbose=0)
rf_search.fit(X_train_tfidf, y_train)
print("Best RF Params:", rf_search.best_params_)
y_pred_rf = rf_search.predict(X_test_tfidf)
print(classification_report(y_test, y_pred_rf))

print("\n=== SVM TUNING (FAST) ===")
svm = SVC()
svm_params = {
    'C': [1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale']
}

# DEEP LEARNING WITH LSTM
print("\n=== LSTM MODEL ===")
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['clean_text'])

X_seq = tokenizer.texts_to_sequences(df['clean_text'])
X_pad = pad_sequences(X_seq, maxlen=50)

label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(df['sentiment'])
y_cat = to_categorical(y_encoded)

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_pad, y_cat, test_size=0.2, stratify=y_cat, random_state=42)

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=50),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_dl, y_train_dl, validation_split=0.1, epochs=5, batch_size=32, verbose=1)
loss, acc = model.evaluate(X_test_dl, y_test_dl, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# Classification report
y_pred_probs = model.predict(X_test_dl)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test_dl, axis=1)
print(classification_report(y_true_classes, y_pred_classes, target_names=label_enc.classes_))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



=== LOGISTIC REGRESSION ===
              precision    recall  f1-score   support

    negative       0.77      0.85      0.81      5590
     neutral       0.79      0.69      0.74      4527
    positive       0.80      0.79      0.80      5164

    accuracy                           0.78     15281
   macro avg       0.78      0.78      0.78     15281
weighted avg       0.78      0.78      0.78     15281


=== RANDOM FOREST TUNING (FAST) ===
Best RF Params: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': None}
              precision    recall  f1-score   support

    negative       0.91      0.90      0.90      5590
     neutral       0.91      0.85      0.88      4527
    positive       0.85      0.92      0.88      5164

    accuracy                           0.89     15281
   macro avg       0.89      0.89      0.89     15281
weighted avg       0.89      0.89      0.89     15281


=== SVM TUNING (FAST) ===

=== LSTM MODEL ===
Epoch 1/5
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 34s